## Задание 1
Для датафрейма `visit_log` из материалов занятия создайте столбец `source_type` по правилам:

- если источник `traffic_source` равен `Yandex` или `Google`, то в `source_type` ставится `organic`;
- для источников `paid` и `email` из России ставим `ad`;
- для источников `paid` и `email` не из России ставим `other`;
- все остальные варианты берём из `traffic_source` без изменений.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

def source_type_update(log):
    log['source_type'] = log['traffic_source'].copy()

    log.loc[log['traffic_source'].isin(['yandex', 'google']), 'source_type'] = 'organic'
    log.loc[log['traffic_source'].isin(['paid', 'email']) & (log['region'] == 'Russia'), 'source_type'] = 'ad'
    log.loc[log['traffic_source'].isin(['paid', 'email']) & (log['region'] != 'Russia'), 'source_type'] = 'other'

    return log

log = pd.read_csv('/content/drive/MyDrive/PYDA/PYDA_HW10/visit_log.csv', sep=';')
updated_log = source_type_update(log)
updated_log

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic
...,...,...,...,...,...,...,...
18933,1550094288,57e5ba8560,https://host.ru/c2382eb3d6afc8d0f3,Belarus,98b19810d0,paid,other
18934,1550094296,6f9389ec1b,https://host.ru/f1eb4601740d627ab0,Russia,32ebb20c13,paid,ad
18935,1550094308,e8cf2eb8e6,https://host.ru/a5dda93e70318570c0,Belarus,b85baa8c73,yandex,organic
18936,1550094314,79530b9a67,https://host.ru/6fda01ec57f23abc9e,Russia,e154b06121,paid,ad


## Задание 2
В файле `URLs.txt` содержатся URL страниц новостного сайта. Вам нужно отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри URL конструкцию: /, затем 8 цифр, затем дефис. Выполните действия:

1. Прочитайте содержимое файла с датафрейм.
2. Отфильтруйте страницы с текстом новостей, используя метод `str.contains` и регулярное выражение в соответствие с заданным шаблоном.

In [ ]:
import pandas as pd


def find_news(urls, pattern):
  # Находим в файле строки по шаблону
  mask = urls.loc[urls['url'].str.contains(pattern, case=False)]
  return mask


urls = pd.read_table('/content/drive/MyDrive/PYDA/PYDA_HW10/URLs.txt')
pattern = '/[0-9]{8}-'

find_news(urls, pattern)

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...
...,...
88,/cis/35984145-kreml-prokommentiroval-soobschen...
89,/video/36071019-olimpiyskie-obekty-rio-prevrat...
90,/science/36151301-nazvano-posledstvie-zloupotr...
91,/incidents/36027330-vospitatelnitsu-zatravili-...


## Задание 3
Используйте файл с оценками фильмов `ratings.csv`. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значениями столбца `timestamp` для данного значения `userId`.

In [ ]:
import pandas as pd

ratings = pd.read_csv('/content/drive/MyDrive/PYDA/PYDA_HW10/ml-latest-small/ratings.csv')


def avg_lifetime(ratings):
    # Группировка данных по userId и подсчет количества оценок
    users_count = ratings.groupby('userId')['rating'].count()

    # Выбор пользователей, которые выставили более 100 оценок
    power_users = users_count[users_count > 100].index

    # Фильтрация данных только для мощных пользователей
    filtered_df = ratings[ratings['userId'].isin(power_users)]

    # Вычисление времени жизни для каждого пользователя
    user_lifetime = filtered_df.groupby('userId')['timestamp'].apply(lambda x: x.max() - x.min())

    # Вычисление среднего времени жизни для мощных пользователей
    average_lifetime = user_lifetime.mean()

    return average_lifetime


print('Среднее время жизни пользователей, выставивших более 100 оценок:', avg_lifetime(ratings))

Среднее время жизни пользователей, выставивших более 100 оценок: 40080507.4496124


## Задание 4
Дана статистика услуг перевозок клиентов компании по типам (см. файл `Python_13_join.ipynb` в разделе «Материалы для лекции “Продвинутый pandas”» ---- Ноутбуки к лекции «Продвинутый pandas»).
Нужно сформировать две таблицы:

- таблицу с тремя типами выручки для каждого `client_id` без указания адреса клиента;
- аналогичную таблицу по типам выручки с указанием адреса клиента.
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

### К домашнему заданию №4
Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [ ]:
import pandas as pd

rzd = pd.DataFrame({'client_id': [111, 112, 113, 114, 115],
                    'rzd_revenue': [1093, 2810, 10283, 5774, 981]})

auto = pd.DataFrame({'client_id': [113, 114, 115, 116, 117],
                     'auto_revenue': [57483, 83, 912, 4834, 98]})

air = pd.DataFrame({'client_id': [115, 116, 117, 118],
                    'air_revenue': [81, 4, 13, 173]})

client_base = pd.DataFrame({'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
                            'address': ['Комсомольская 4', 'Энтузиастов 8а',
                                        'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1',
                                        'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']})


revenue = pd.merge(pd.merge(rzd, auto, on='client_id', how='outer'), air, on='client_id', how='outer')
revenue_with_address = pd.merge(revenue, client_base, on='client_id')

print('Таблица без адреса клиента:\n', revenue)
print('\nТаблица с адресом клиента:\n', revenue_with_address)

Таблица без адреса клиента:
    client_id  rzd_revenue  auto_revenue  air_revenue
0        111       1093.0           NaN          NaN
1        112       2810.0           NaN          NaN
2        113      10283.0       57483.0          NaN
3        114       5774.0          83.0          NaN
4        115        981.0         912.0         81.0
5        116          NaN        4834.0          4.0
6        117          NaN          98.0         13.0
7        118          NaN           NaN        173.0

Таблица с адресом клиента:
    client_id  rzd_revenue  auto_revenue  air_revenue          address
0        111       1093.0           NaN          NaN  Комсомольская 4
1        112       2810.0           NaN          NaN   Энтузиастов 8а
2        113      10283.0       57483.0          NaN  Левобережная 1а
3        114       5774.0          83.0          NaN          Мира 14
4        115        981.0         912.0         81.0        ЗЖБИиДК 1
5        116          NaN        4834.0      